In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import f1_score, classification_report
import gc # 垃圾回收import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter # 用于检查类别分布
# 💡 新增：导入 SMOTENC (用于处理类别特征 ticker_id)
from imblearn.over_sampling import SMOTENC

# 设置 Matplotlib/Seaborn 样式
sns.set(style='whitegrid')
# 设置 Pandas 显示选项
pd.set_option('display.max_columns', None)

In [2]:
print("正在加载特征化训练数据 (train_with_features.csv)...")
try:
    df_train = pd.read_csv('train_with_features.csv', low_memory=False)
    print(f"数据加载成功！形状: {df_train.shape}")
except FileNotFoundError:
    print("错误：train_with_features.csv 文件未找到。")
    raise# 恢复数据类型
print("正在恢复数据类型...")
df_train['t'] = pd.to_datetime(df_train['t'])
df_train['ticker_id'] = df_train['ticker_id'].astype('category')
print("数据类型恢复完毕。")

正在加载特征化训练数据 (train_with_features.csv)...
数据加载成功！形状: (1932, 68736)
正在恢复数据类型...
数据类型恢复完毕。


In [3]:
# 1. 定义 5 分类目标映射
target_map = {'HH': 0, 'HL': 1, 'LH': 2, 'LL': 3}
df_train['class_label_encoded'] = df_train['class_label'].map(target_map)
df_train['class_label_encoded'] = df_train['class_label_encoded'].fillna(4)
df_train['class_label_encoded'] = df_train['class_label_encoded'].astype(int)

print("目标变量编码完成。")
print(df_train['class_label_encoded'].value_counts().sort_index())

目标变量编码完成。
class_label_encoded
0      40
1      28
2      18
3      26
4    1820
Name: count, dtype: int64


In [4]:
# 1. 定义目标和元数据
TARGET = 'class_label_encoded'
METADATA_COLS = ['class_label', 'class_label_encoded', 't', 'id', 'train_id']

# 2. 加载 V2 模型的特征重要性
try:
    importance_df = pd.read_csv('feature_importance_v2.csv')
except FileNotFoundError:
    print("错误：'feature_importance_v2.csv' 未找到！")
    print("请返回 03_Model_v2... 笔记本，运行 CELL 8 以保存该文件。")
    raise# 3. 定义我们的“精英特征集” (与 V3 相同)
NUM_TOP_FEATURES = 300
top_features = importance_df.head(NUM_TOP_FEATURES)['feature'].tolist()

# 4. 确保 'ticker_id' 被包含在内
if 'ticker_id' not in top_features:
    top_features.append('ticker_id')

# 5. 将此列表设置为我们的最终 FEATURES
FEATURES = top_features

print(f"--- Model v6 (SMOTE) 特征选择 ---")
print(f"SMOTE 将在 {len(FEATURES)} 个 Top 特征上运行。")

--- Model v6 (SMOTE) 特征选择 ---
SMOTE 将在 300 个 Top 特征上运行。


In [5]:
# 1. 确保数据严格按时间排序
print("正在按 't' 和 'ticker_id' 排序数据以进行验证分割...")
df_train = df_train.sort_values(by=['t', 'ticker_id']).reset_index(drop=True)

# 2. 定义 80/20 分割点
split_percentage = 0.8
split_index = int(len(df_train) * split_percentage)

# 3. 分割数据
train_data = df_train.iloc[:split_index]
val_data = df_train.iloc[split_index:]
print(f"新训练集样本数: {len(train_data)}, 新验证集样本数: {len(val_data)}")

# 4. 创建 X (特征) 和 y (目标) (只使用 Top 300 特征)
X_train = train_data[FEATURES]
y_train = train_data[TARGET]
X_val = val_data[FEATURES]
y_val = val_data[TARGET]

# 5. 修复数据类型 和 填充 NaN
print("正在修复 'object' 数据类型并填充 NaN...")
# 找出 'ticker_id' 之外的所有特征列
numeric_features = [col for col in FEATURES if col != 'ticker_id']

# 我们使用 .copy() 来避免 Pandas 的 SettingWithCopyWarning
X_train_copy = X_train.copy()
X_val_copy = X_val.copy()

for col in numeric_features:
    # 强制转换为数值类型
    X_train_copy[col] = pd.to_numeric(X_train_copy[col], errors='coerce', downcast='float')
    X_val_copy[col] = pd.to_numeric(X_val_copy[col], errors='coerce', downcast='float')

# 替换回原来的变量
X_train = X_train_copy
X_val = X_val_copy
print(f"已将 {len(numeric_features)} 个特征列强制转换为数值类型。")

# ----------------------------------------------------
# 💡 【错误修复】 只对 numeric_features 列进行 fillna
# ----------------------------------------------------
print("正在用 -999 填充剩余的 NaN 值 (仅数值列)...")
X_train[numeric_features] = X_train[numeric_features].fillna(-999)
X_val[numeric_features] = X_val[numeric_features].fillna(-999)
print("NaN 填充完毕！")
# ----------------------------------------------------

# 6. 清理内存
del df_train, train_data, val_data, X_train_copy, X_val_copy
gc.collect()
print("验证集创建完毕！")

正在按 't' 和 'ticker_id' 排序数据以进行验证分割...
新训练集样本数: 1545, 新验证集样本数: 387
正在修复 'object' 数据类型并填充 NaN...
已将 299 个特征列强制转换为数值类型。
正在用 -999 填充剩余的 NaN 值 (仅数值列)...
NaN 填充完毕！
验证集创建完毕！


In [6]:
print("--- 原始训练集 (X_train) 分布 ---")
print(Counter(y_train))

# 1. 找到 'ticker_id' 的列索引
# SMOTENC 需要知道哪一列是类别特征
try:
    categorical_feature_index = [X_train.columns.get_loc('ticker_id')]
    print(f"\n'ticker_id' 在特征中的索引为: {categorical_feature_index[0]}")
except KeyError:
    print("\n错误：'ticker_id' 不在 X_train 的列中！")
    raise
# 2. 定义 SMOTENC 策略
# 'sampling_strategy'='auto' (或 'not majority') 
# 将把所有稀有类 (0,1,2,3) 的样本数量提升到与多数类 (4) 相同
smote_nc = SMOTENC(
    sampling_strategy='auto',
    categorical_features=categorical_feature_index,
    random_state=42
)

print("\n正在应用 SMOTENC (只对训练集)...")
print("(这可能需要几分钟，因为它正在 '创造' 新的合成样本...)")
X_train_smote, y_train_smote = smote_nc.fit_resample(X_train, y_train)

# 3. 检查新的平衡后分布
print("\n--- SMOTE (过采样后) 训练集分布 ---")
print(Counter(y_train_smote))
print(f"原始 X_train 形状: {X_train.shape}")
print(f"SMOTE 后 X_train_smote 形状: {X_train_smote.shape}")

--- 原始训练集 (X_train) 分布 ---
Counter({4: 1461, 0: 30, 3: 21, 1: 19, 2: 14})

'ticker_id' 在特征中的索引为: 168

正在应用 SMOTENC (只对训练集)...
(这可能需要几分钟，因为它正在 '创造' 新的合成样本...)

--- SMOTE (过采样后) 训练集分布 ---
Counter({4: 1461, 0: 1461, 3: 1461, 2: 1461, 1: 1461})
原始 X_train 形状: (1545, 300)
SMOTE 后 X_train_smote 形状: (7305, 300)


In [7]:
# 1. 定义 LightGBM 模型参数
lgb_params = {
    'objective': 'multiclass',
    'num_class': 5,
    'metric': 'multi_logloss',
    'n_estimators': 500,
    'learning_rate': 0.05,
    'n_jobs': -1,
    'seed': 42,
    # 移除 'class_weight': 'balanced'
}

# 2. 初始化模型
model_v6 = lgb.LGBMClassifier(**lgb_params)

# 3. 训练模型
print(f"开始训练 Model v6 (使用 SMOTE 数据)...")
# (注意：X_val 仍然是原始的、未采样的 300 特征验证集)
model_v6.fit(
    X_train_smote, y_train_smote,        # 👈 使用 SMOTE 后的数据
    eval_set=[(X_val, y_val)],            # 👈 评估仍使用 *原始* 验证集
    eval_metric='multi_logloss',
    callbacks=[lgb.early_stopping(50)],
    categorical_feature=['ticker_id']
)

print("Model v6 训练完成！")

开始训练 Model v6 (使用 SMOTE 数据)...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008154 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 17054
[LightGBM] [Info] Number of data points in the train set: 7305, number of used features: 276
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[55]	valid_0's multi_logloss: 0.380834
Model v6 训练完成！


In [8]:
# 1. 在验证集上进行预测
print("正在验证集上进行预测...")
val_preds = model_v6.predict(X_val)

# 2. 计算 Macro F1-Score
macro_f1_v6 = f1_score(y_val, val_preds, average='macro', zero_division=0)

print("\n========================================")
print(f"📈 Model v6 (SMOTE) Macro F1-Score: {macro_f1_v6:.4f}")
print("----------------------------------------")
print(f"📈 Model v2 (All Features) Macro F1-Score: 0.2238")
print(f"📈 Model v4 (Undersampled 1:1) F1: 0.2250")
print(f"📉 Model v5 (Undersampled 1:10) F1: 0.1925")
print(f"📉 Model v3 (Top 300) F1: 0.1916")
print(f"📉 Baseline F1: 0.1905")
print("========================================")
print(f"🚀 V6 vs V2 (Best) 性能提升: {macro_f1_v6 - 0.2238:.4f}")
print("========================================")

# 3. 打印详细的分类报告
target_map_full = {'HH': 0, 'HL': 1, 'LH': 2, 'LL': 3, 'None': 4}
target_names = [key for key, val in sorted(target_map_full.items(), key=lambda item: item[1])]
all_labels = [val for key, val in sorted(target_map_full.items(), key=lambda item: item[1])]

print("\n详细分类报告 (Model v6):")
print(classification_report(
    y_val, 
    val_preds, 
    target_names=target_names, 
    labels=all_labels,
    zero_division=0
))

正在验证集上进行预测...

📈 Model v6 (SMOTE) Macro F1-Score: 0.1917
----------------------------------------
📈 Model v2 (All Features) Macro F1-Score: 0.2238
📈 Model v4 (Undersampled 1:1) F1: 0.2250
📉 Model v5 (Undersampled 1:10) F1: 0.1925
📉 Model v3 (Top 300) F1: 0.1916
📉 Baseline F1: 0.1905
🚀 V6 vs V2 (Best) 性能提升: -0.0321

详细分类报告 (Model v6):
              precision    recall  f1-score   support

          HH       0.00      0.00      0.00        10
          HL       0.00      0.00      0.00         9
          LH       0.00      0.00      0.00         4
          LL       0.00      0.00      0.00         5
        None       0.93      0.99      0.96       359

    accuracy                           0.92       387
   macro avg       0.19      0.20      0.19       387
weighted avg       0.86      0.92      0.89       387

